# Creating the Input Database (InputDB) for ecPoint-Calibrate

### What does this Jupyter notebook?
This Jupyter notebook will generate the input database, for forecasts and observations, in the format required by ecPoint-Calibrate.

### What raw forecasts will be used?
The raw forecasts that will be used in this Jupyter notebook come from form the Integrated Forecasting System (IFS) of the European Centre for Medium-range Weather Forecasts (ECMWF). The forecasts come from an experiment run to create forecasts with the the 47r1 version of the IFS, for a period between January 1st and December 31st, 2019. The forecasts are provided in grib format.

### What raw observations will be used used?
The raw observations used in this software are the "Global Surface Summary of Day" product produced by the NOAA's National Centers for Environmental Information (NCEI). The observations are freely downloadable from https://www.ncei.noaa.gov/data/global-summary-of-the-day/archive/ and are processed with pandas and Metview (https://pypi.org/project/metview/). From all the parameters in the raw files, this Jupyter notebook will use only:
1. Precipitation amount for 24 hours, PRCP (.01 inches)
2. Mean temperature over 24 hours, TEMP (.1 Fahrenheit)
The observations will be converted in geopoints, and the units will get converted in mm for precipitation, and Celsius degrees for temperature.

NOTE: each file contains all the observations for the year at a specific station

______________________________
## Set the environment

In [1]:
import os
import os.path
from os import path
import fileinput
import tarfile
from datetime import date, datetime, timedelta
import numpy as np
import pandas as pd
import metview as mv

In [2]:
# INPUT PARAMETERS
DateS = date(2019, 1, 1)
DateF = date(2019, 1, 1)
Delta_Date = timedelta(days=1)
WorkDir = os.path.dirname(os.getcwd())
RawData_Dir = "RawData"
InputDB_Dir = "InputDB" 
FC_Dir = "FC"
OBS_Dir = "OBS"

In [3]:
# CREATING SOME ENVIRONMENT VARIABLES

# Raw data and input database directories
RawData_Dir = WorkDir + "/" + RawData_Dir
InputDB_Dir = WorkDir + "/" + InputDB_Dir

# Tared raw forecasts (FC) and observations (OBS) files
FC_zip = RawData_Dir + "/" + FC_Dir + ".tar.gz"
OBS_zip = RawData_Dir + "/" + OBS_Dir + ".tar.gz"

# Untared raw FC and OBS files
RawData_FC_Dir = RawData_Dir + "/" + FC_Dir
RawData_OBS_Dir = RawData_Dir + "/" + OBS_Dir

# Input database for ecPoint-Calibrate
InputDB_prcp_FC_Dir = InputDB_Dir + "/Rainfall/" + FC_Dir
InputDB_prcp_OBS_Dir = InputDB_Dir + "/Rainfall/" + OBS_Dir
InputDB_temp_FC_Dir = InputDB_Dir + "/Temperature/" + FC_Dir
InputDB_temp_OBS_Dir = InputDB_Dir + "/Temperature/" + OBS_Dir

_________________________________________________________________________
## Get the raw forecasts and observations (from Zenodo)

In [4]:
%%bash

# DOWNLOAD THE RAW DATA FROM ZENODO 
# Note: it can take up to 1 hour

RawData_ZenodoDOI="10.5281/zenodo.4642836" 
cd ./../RawData
if [[ -f "FC.tar.gz" && -f "OBS.tar.gz"   ]]; then
    echo "Raw FC and OBS have already been downloaded from Zenodo."
else
    echo "Downloading raw FC and OBS from Zenodo..."
    zenodo_get ${RawData_ZenodoDOI}
fi
rm -rf md5sums.txt

Raw FC and OBS have already been downloaded from Zenodo.


In [5]:
# UNTAR RAW DATA

# Observations
# Note: it can take up to 5 minutes
if path.exists(RawData_Dir + "/OBS") == True:
    print("Raw OBS has already been untared.")
else:
    print("Extracting the contents of " + OBS_zip + " ...")
    tar = tarfile.open(OBS_zip)
    tar.extractall(path=RawData_Dir)
    tar.close()

# Forecasts
# Note: it can take up to 30 minutes
if path.exists(RawData_Dir + "/FC") == True:
    print("Raw FC has already been untared.")
else:
    print("Extracting the contents of " + FC_zip + " ...")
    tar = tarfile.open(FC_zip)
    tar.extractall(path=RawData_Dir)
    tar.close()

Raw OBS has already been untared.
Raw FC has already been untared.


________________________________________________________________
## Create the input database for ecPoint-Calibrate

In [ ]:
# CREATE THE INPUT DATABASE FOR OBS
# Note: it can take up to 3 hours

# List all the raw observation files in the directory "RawData_OBS_Dir"
arr = os.listdir(RawData_OBS_Dir)
print("NOTE:")
print("There are " + str(len(arr)) + " stations around the globe to analyse.")

# Creation of the InputDB for OBS
TheDate = DateS
while TheDate <= DateF:
    
    TheDateSTR = TheDate.strftime("%Y-%m-%d")
    print(" ")
    print("Creating OBS's InputDB for day " + TheDateSTR)
    
    
    ##################################################
    # MERGING GLOBAL OBSERVATIONS FOR CONSIDERED DAY #
    ##################################################
    
    # Generating empty dataframes for the considered day
    prcp = pd.DataFrame()
    temp = pd.DataFrame()
    
    for RawData_OBS_Filename in arr:
        
        # Read the raw observations for each station
        RawData_OBS_File = RawData_OBS_Dir + "/" + RawData_OBS_Filename
        df = pd.read_csv(RawData_OBS_File)
        df1 = df[df["DATE"].isin([TheDateSTR])] #selection of the date of interest
        
        # Selecting the variables of interest for precipitation
        prcp1 = df1[["STATION", "LATITUDE", "LONGITUDE", "ELEVATION", "DATE", "PRCP", "PRCP_ATTRIBUTES"]]
        frames = [prcp, prcp1]
        prcp = pd.concat(frames)
          
        # Selecting the variables of interest for temperature
        temp1 = df1[["STATION", "LATITUDE", "LONGITUDE", "ELEVATION", "DATE", "TEMP", "TEMP_ATTRIBUTES"]]
        frames = [temp, temp1]
        temp = pd.concat(frames)

        
    ###################################
    # PRE-PROCESSING THE OBSERVATIONS #
    ###################################
    
    # Precipitation observations
    prcp = prcp[prcp.PRCP != 99.99] # eliminating the missing values
    prcp = prcp[prcp.PRCP_ATTRIBUTES != "A"] # eliminating the stations that reported only 1 report of 6-hour precipitation amount.
    prcp = prcp[prcp.PRCP_ATTRIBUTES != "B"] # eliminating the stations that reported only the summation of 2 reports of 6-hour precipitation amount.
    prcp = prcp[prcp.PRCP_ATTRIBUTES != "C"] # eliminating the stations that reported only the summation of 3 reports of 6-hour precipitation amount.
    prcp = prcp[prcp.PRCP_ATTRIBUTES != "E"] # eliminating the stations that reported only 1 report of 12-hour precipitation amount.
    prcp = prcp[prcp.PRCP_ATTRIBUTES != "H"] # eliminating the stations that reported incomplete data for the day.
    prcp = prcp[prcp.PRCP_ATTRIBUTES != "I"] # eliminating the stations that did not report any precipitation data for the day.
    prcp["PRCP"] = prcp["PRCP"] * 25.4 # converting rainfall in mm from inches.
    del prcp['PRCP_ATTRIBUTES'] # eliminate the "PRCS_ATTRIBUTES" column from the final output
    print("Total n. of rainfall observations maintained: " + str(len(prcp)))
    
    # Temperature observations
    temp = temp[temp.TEMP != 9999.9] # eliminating the missing values.
    temp = temp[temp.TEMP_ATTRIBUTES >= 20] # eliminating those reports that did not provide hourly observations for at least 20 hours in the day.
    temp["TEMP"] = (temp["TEMP"] - 32) * (5/9) # converting temperature in Celsius from Farenheit degrees.
    del temp['TEMP_ATTRIBUTES'] # eliminate the "TEMP_ATTRIBUTES" column from the final output
    print("Total n. of temperature observations maintained: " + str(len(temp)))
    
    
    ###########################
    # CREATING GEOPOINT FILES #
    ###########################
    
    # The files are named with the date and time of the end of the period the observations refer to
    TheDate_File = TheDate + timedelta(days=1)
    TheDateSTR_File = TheDate_File.strftime("%Y%m%d")
    
    # -------------------------- #
    # Precipitation observations #
    # -------------------------- #
    
    # Create the InputDB directory for the considered date if it doesn't exist
    InputDB_prcp_OBS_Dir_temp = InputDB_prcp_OBS_Dir + "/Acc24h/" + TheDateSTR_File
    if not os.path.exists(InputDB_prcp_OBS_Dir_temp): 
        os.makedirs(InputDB_prcp_OBS_Dir_temp)
    
    # Create the geopoint file (with Metview)
    prcp_geo = mv.create_geo(type = "standard",
                             latitudes = prcp["LATITUDE"].to_numpy() ,
                             longitudes = prcp["LONGITUDE"].to_numpy(),
                             levels = prcp["ELEVATION"].to_numpy(),
                             values = prcp["PRCP"].to_numpy(),
                             dates = int(TheDateSTR_File),
                             times = 0)                    
                    
    # Saving the geopoint file
    InputDB_prcp_OBS_File = InputDB_prcp_OBS_Dir_temp + "/tp_24_" + TheDateSTR_File + "_00.geo"
    mv.write(InputDB_prcp_OBS_File, prcp_geo)
    
    
    # ------------------------ #
    # Temperature observations #
    # ------------------------ #
    
    # Create the InputDB directory for the considered date if it doesn't exist
    InputDB_temp_OBS_Dir_temp = InputDB_temp_OBS_Dir + "/" + TheDateSTR_File
    if not os.path.exists(InputDB_temp_OBS_Dir_temp): 
        os.makedirs(InputDB_temp_OBS_Dir_temp)
    
    # Create the geopoint file (with Metview)
    temp_geo = mv.create_geo(type = "standard",
                            latitudes = temp["LATITUDE"].to_numpy() ,
                            longitudes = temp["LONGITUDE"].to_numpy(),
                            levels = temp["ELEVATION"].to_numpy(),
                            values = temp["TEMP"].to_numpy(),
                            dates = int(TheDateSTR_File),
                            times = 0)                    
                    
    # Saving the geopoint file
    InputDB_temp_OBS_File = InputDB_temp_OBS_Dir_temp + "/t_" + TheDateSTR_File + "_00.geo"
    mv.write(InputDB_temp_OBS_File, temp_geo)
    
    
    TheDate += Delta_Date

________________________________________________________________
## Plot forecasts and observations (with Metview)

In [ ]:
# Plot rainfall observations
InputDB_prcp_OBS_File = "/home/jovyan/InputDB/Rainfall/OBS/20190102/2019010200.geo"
print(InputDB_prcp_OBS_File)
g = mv.read(InputDB_prcp_OBS_File)

# To display plots within the Jupyter notebook
mv.setoutput("jupyter")

my_coast = mv.mcoast(
    map_coastline_land_shade        = "on",
    map_coastline_land_shade_colour = "RGB(0.89,0.89,0.89)",
    map_coastline_sea_shade         = "on",
    map_coastline_sea_shade_colour  = "grey",
    map_grid_latitude_increment     = 20,
    map_grid_longitude_increment    = 40,
    map_grid_colour                 = "charcoal"
    )

my_symbol = mv.msymb(
    legend                               = "on",
    symbol_type                          = "marker",
    symbol_marker_index                  = 15,
    symbol_table_mode                    = "advanced",
    symbol_outline                       = "on",
    symbol_advanced_table_selection_type = "list",
    symbol_advanced_table_level_list     = [0,0.001,1,2,5,10,30,50,100,200],
    symbol_advanced_table_colour_method  = "list",
    symbol_advanced_table_colour_list    = ["rgb(204, 230, 255)","rgb(0, 136, 204)","rgb(0, 255, 191)","rgb(204, 255, 153)","rgb(255, 255, 102)","rgb(255, 204, 153)","rgb(255, 102, 0)","rgb(204, 0, 102)","rgb(102, 0, 102)"],
    symbol_advanced_table_height_list    = 0.5
)

# Plot
mv.plot(my_coast,my_symbol,g)